In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from skimage.io import imsave
import os
import shutil
import gzip
img_height = 28
img_width = 28
img_size = img_height * img_width

to_train = True
to_restore = False
output_path = "output"

max_epoch = 2

h1_size = 150
h2_size = 300
z_size = 100
batch_size = 256


/Users/munsuyeong/Downloads/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_mnist(path, kind='train'):

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [3]:
# 제너레이터 (G)

    # ----------- 코드 작성부분 ----------------

def generator(z):
    w1 = tf.Variable(tf.truncated_normal([z_size, h1_size],stddev=0.1),name="g_wl", dtype=tf.float32)
    b1 = tf.Variable(tf.zeros([h1_size]),name="g_b1", dtype=tf.float32)
    h1 = tf.nn.relu(tf.matmul(z,w1)+b1)
        
    w2 = tf.Variable(tf.truncated_normal([h1_size, h2_size],stddev=0.1),name="g_w2", dtype=tf.float32)
    b2 = tf.Variable(tf.zeros([h2_size]),name="g_b2", dtype=tf.float32)
    h2 = tf.nn.relu(tf.matmul(h1,w2)+b2)
        
    w3 = tf.Variable(tf.truncated_normal([h2_size, img_size],stddev=0.1),name="g_w3", dtype=tf.float32)
    b3 = tf.Variable(tf.zeros([img_size]),name="g_b3", dtype=tf.float32)
    h3 = tf.matmul(h2,w3)+b3
    
    g_params = [w1,b1,w2,b2,w3,b3]
    x_generate = tf.nn.tanh(h3)
       
    return x_generate, g_params
    # ---------------------------------------

In [4]:
# 디스크리미네이터 (D)

    # ----------- 코드 작성부분 ----------------

def discriminator(x_data, x_generated, keep_prob):
    x_in = tf.concat([x_data, x_generated],0)
            
    w1 = tf.Variable(tf.truncated_normal([img_size,h2_size],stddev=0.1),name='d_w1', dtype= tf.float32)
    b1 = tf.Variable(tf.zeros([h2_size]),name="d_b1", dtype=tf.float32)
    h1 = tf.nn.dropout(tf.nn.relu(tf.matmul(x_in,w1)+b1),keep_prob)
        
    w2 = tf.Variable(tf.truncated_normal([h2_size,h1_size],stddev=0.1),name='d_w2', dtype= tf.float32)
    b2 = tf.Variable(tf.zeros([h1_size]),name="d_b2", dtype=tf.float32)
    h2 = tf.nn.dropout(tf.nn.relu(tf.matmul(h1,w2)+b2),keep_prob)
        
    w3 = tf.Variable(tf.truncated_normal([h1_size,1],stddev=0.1),name='d_w3', dtype= tf.float32)
    b3 = tf.Variable(tf.zeros([1]),name="d_b3", dtype=tf.float32)
    h3 = tf.matmul(h2,w3)+b3
        
    y_data = tf.nn.sigmoid(tf.slice(h3,[0,0],[batch_size,-1],name=None))
    y_generated = tf.nn.sigmoid(tf.slice(h3,[batch_size,0],[-1,-1],name=None))
        
    d_params = [w1,b1,w2,b2,w3,b3]
        
    return y_data, y_generated, d_params
    # ---------------------------------------
    
    

In [5]:
def show_result(batch_res, fname, grid_size=(8, 8), grid_pad=5):
    batch_res = 0.5 * batch_res.reshape((batch_res.shape[0], img_height, img_width)) + 0.5
    img_h, img_w = batch_res.shape[1], batch_res.shape[2]
    grid_h = img_h * grid_size[0] + grid_pad * (grid_size[0] - 1)
    grid_w = img_w * grid_size[1] + grid_pad * (grid_size[1] - 1)
    img_grid = np.zeros((grid_h, grid_w), dtype=np.uint8)
    for i, res in enumerate(batch_res):
        if i >= grid_size[0] * grid_size[1]:
            break
        img = (res) * 255.
        img = img.astype(np.uint8) 
        row = (i // grid_size[0]) * (img_h + grid_pad)
        col = (i % grid_size[1]) * (img_w + grid_pad)
        img_grid[row:row + img_h, col:col + img_w] = img
    imsave(fname, img_grid)

In [ ]:
def train():
    
    global_step = tf.Variable(0, name="global_step", trainable=False)
    
    
    # ----------- 코드 작성부분 ----------------
    
    train_x, train_y = load_mnist("fashion_mnist",kind='train')
    size = train_x.shape[0]
    x_data = tf.placeholder(tf.float32, [batch_size,img_size],name="x_data")
    z=tf.placeholder(tf.float32, [batch_size,z_size],name="z")
    keep_prob = tf.placeholder(tf.float32,name="keep_prob")
    
    x_generated, g_params = generator(z)
    
    y_data, y_generated, d_params = discriminator(x_data, x_generated, keep_prob)

    # ---------------------------------------

    d_loss = -(tf.log(y_data)+tf.log(1-y_generated))
    g_loss = -tf.log(y_generated)
    
    optimizer = tf.train.AdamOptimizer(0.0001)
    
    d_trainer = optimizer.minimize(d_loss,var_list=d_params)
    g_trainer = optimizer.minimize(g_loss,var_list=g_params)
    
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    
    sess = tf.Session()
    
    if to_restore:
        chkpt_fname = tf.train.latest_checkpoint(output_path)
        saver.restore(sess, chkpt_fname)
    else:
        if os.path.exists(output_path):
            shutil.rmtree(output_path)
        os.mkdir(output_path)

    z_sample_val = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)

    sess = tf.Session()
        
    sess.run(init)
    
    for i in range(sess.run(global_step), max_epoch):
        for j in range(21870 // batch_size):
            print("epoch:%s, iter:%s" % (i, j))

            batch_end = j * batch_size + batch_size
            if batch_end >= size:
                batch_end = size - 1
            x_value = train_x[ j * batch_size : batch_end ]
            x_value = x_value / 255.
            x_value = 2 * x_value - 1

            z_value = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)
            sess.run(d_trainer,
                     feed_dict={x_data: x_value, z: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
            if j % 1 == 0:
                sess.run(g_trainer,
                         feed_dict={x_data: x_value, z: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
        x_gen_val = sess.run(x_generated, feed_dict={z: z_sample_val})
        show_result(x_gen_val, os.path.join(output_path, "sample%s.jpg" % i))
        sess.run(tf.assign(global_step, i + 1))
        
    saver.save(sess, 'output/gdg_codlab_study.ckpt')

In [ ]:
train()

epoch:0, iter:0
epoch:0, iter:1
epoch:0, iter:2
epoch:0, iter:3
epoch:0, iter:4
epoch:0, iter:5
epoch:0, iter:6
epoch:0, iter:7
epoch:0, iter:8
epoch:0, iter:9
epoch:0, iter:10
epoch:0, iter:11
epoch:0, iter:12
epoch:0, iter:13
epoch:0, iter:14
epoch:0, iter:15
epoch:0, iter:16
epoch:0, iter:17
epoch:0, iter:18
epoch:0, iter:19
epoch:0, iter:20
epoch:0, iter:21
epoch:0, iter:22
epoch:0, iter:23
epoch:0, iter:24
epoch:0, iter:25
epoch:0, iter:26
epoch:0, iter:27
epoch:0, iter:28
epoch:0, iter:29
epoch:0, iter:30
epoch:0, iter:31
epoch:0, iter:32
epoch:0, iter:33
epoch:0, iter:34
epoch:0, iter:35
epoch:0, iter:36
epoch:0, iter:37
epoch:0, iter:38
epoch:0, iter:39
epoch:0, iter:40
epoch:0, iter:41
epoch:0, iter:42
epoch:0, iter:43
epoch:0, iter:44
epoch:0, iter:45
epoch:0, iter:46
epoch:0, iter:47
epoch:0, iter:48
epoch:0, iter:49
epoch:0, iter:50
epoch:0, iter:51
epoch:0, iter:52
epoch:0, iter:53
epoch:0, iter:54
epoch:0, iter:55
epoch:0, iter:56
epoch:0, iter:57
epoch:0, iter:58
epoch:0

epoch:5, iter:62
epoch:5, iter:63
epoch:5, iter:64
epoch:5, iter:65
epoch:5, iter:66
epoch:5, iter:67
epoch:5, iter:68
epoch:5, iter:69
epoch:5, iter:70
epoch:5, iter:71
epoch:5, iter:72
epoch:5, iter:73
epoch:5, iter:74
epoch:5, iter:75
epoch:5, iter:76
epoch:5, iter:77
epoch:5, iter:78
epoch:5, iter:79
epoch:5, iter:80
epoch:5, iter:81
epoch:5, iter:82
epoch:5, iter:83
epoch:5, iter:84
epoch:6, iter:0
epoch:6, iter:1
epoch:6, iter:2
epoch:6, iter:3
epoch:6, iter:4
epoch:6, iter:5
epoch:6, iter:6
epoch:6, iter:7
epoch:6, iter:8
epoch:6, iter:9
epoch:6, iter:10
epoch:6, iter:11
epoch:6, iter:12
epoch:6, iter:13
epoch:6, iter:14
epoch:6, iter:15
epoch:6, iter:16
epoch:6, iter:17
epoch:6, iter:18
epoch:6, iter:19
epoch:6, iter:20
epoch:6, iter:21
epoch:6, iter:22
epoch:6, iter:23
epoch:6, iter:24
epoch:6, iter:25
epoch:6, iter:26
epoch:6, iter:27
epoch:6, iter:28
epoch:6, iter:29
epoch:6, iter:30
epoch:6, iter:31
epoch:6, iter:32
epoch:6, iter:33
epoch:6, iter:34
epoch:6, iter:35
epoch:6

epoch:11, iter:31
epoch:11, iter:32
epoch:11, iter:33
epoch:11, iter:34
epoch:11, iter:35
epoch:11, iter:36
epoch:11, iter:37
epoch:11, iter:38
epoch:11, iter:39
epoch:11, iter:40
epoch:11, iter:41
epoch:11, iter:42
epoch:11, iter:43
epoch:11, iter:44
epoch:11, iter:45
epoch:11, iter:46
epoch:11, iter:47
epoch:11, iter:48
epoch:11, iter:49
epoch:11, iter:50
epoch:11, iter:51
epoch:11, iter:52
epoch:11, iter:53
epoch:11, iter:54
epoch:11, iter:55
epoch:11, iter:56
epoch:11, iter:57
epoch:11, iter:58
epoch:11, iter:59
epoch:11, iter:60
epoch:11, iter:61
epoch:11, iter:62
epoch:11, iter:63
epoch:11, iter:64
epoch:11, iter:65
epoch:11, iter:66
epoch:11, iter:67
epoch:11, iter:68
epoch:11, iter:69
epoch:11, iter:70
epoch:11, iter:71
epoch:11, iter:72
epoch:11, iter:73
epoch:11, iter:74
epoch:11, iter:75
epoch:11, iter:76
epoch:11, iter:77
epoch:11, iter:78
epoch:11, iter:79
epoch:11, iter:80
epoch:11, iter:81
epoch:11, iter:82
epoch:11, iter:83
epoch:11, iter:84
epoch:12, iter:0
epoch:12, i

epoch:16, iter:64
epoch:16, iter:65
epoch:16, iter:66
epoch:16, iter:67
epoch:16, iter:68
epoch:16, iter:69
epoch:16, iter:70
epoch:16, iter:71
epoch:16, iter:72
epoch:16, iter:73
epoch:16, iter:74
epoch:16, iter:75
epoch:16, iter:76
epoch:16, iter:77
epoch:16, iter:78
epoch:16, iter:79
epoch:16, iter:80
epoch:16, iter:81
epoch:16, iter:82
epoch:16, iter:83
epoch:16, iter:84
epoch:17, iter:0
epoch:17, iter:1
epoch:17, iter:2
epoch:17, iter:3
epoch:17, iter:4
epoch:17, iter:5
epoch:17, iter:6
epoch:17, iter:7
epoch:17, iter:8
epoch:17, iter:9
epoch:17, iter:10
epoch:17, iter:11
epoch:17, iter:12
epoch:17, iter:13
epoch:17, iter:14
epoch:17, iter:15
epoch:17, iter:16
epoch:17, iter:17
epoch:17, iter:18
epoch:17, iter:19
epoch:17, iter:20
epoch:17, iter:21
epoch:17, iter:22
epoch:17, iter:23
epoch:17, iter:24
epoch:17, iter:25
epoch:17, iter:26
epoch:17, iter:27
epoch:17, iter:28
epoch:17, iter:29
epoch:17, iter:30
epoch:17, iter:31
epoch:17, iter:32
epoch:17, iter:33
epoch:17, iter:34
ep

epoch:22, iter:13
epoch:22, iter:14
epoch:22, iter:15
epoch:22, iter:16
epoch:22, iter:17
epoch:22, iter:18
epoch:22, iter:19
epoch:22, iter:20
epoch:22, iter:21
epoch:22, iter:22
epoch:22, iter:23
epoch:22, iter:24
epoch:22, iter:25
epoch:22, iter:26
epoch:22, iter:27
epoch:22, iter:28
epoch:22, iter:29
epoch:22, iter:30
epoch:22, iter:31
epoch:22, iter:32
epoch:22, iter:33
epoch:22, iter:34
epoch:22, iter:35
epoch:22, iter:36
epoch:22, iter:37
epoch:22, iter:38
epoch:22, iter:39
epoch:22, iter:40
epoch:22, iter:41
epoch:22, iter:42
epoch:22, iter:43
epoch:22, iter:44
epoch:22, iter:45
epoch:22, iter:46
epoch:22, iter:47
epoch:22, iter:48
epoch:22, iter:49
epoch:22, iter:50
epoch:22, iter:51
epoch:22, iter:52
epoch:22, iter:53
epoch:22, iter:54
epoch:22, iter:55
epoch:22, iter:56
epoch:22, iter:57
epoch:22, iter:58
epoch:22, iter:59
epoch:22, iter:60
epoch:22, iter:61
epoch:22, iter:62
epoch:22, iter:63
epoch:22, iter:64
epoch:22, iter:65
epoch:22, iter:66
epoch:22, iter:67
epoch:22, 

...

epoch:251, iter:22
epoch:251, iter:23
epoch:251, iter:24
epoch:251, iter:25
epoch:251, iter:26
epoch:251, iter:27
epoch:251, iter:28
epoch:251, iter:29
epoch:251, iter:30
epoch:251, iter:31
epoch:251, iter:32
epoch:251, iter:33
epoch:251, iter:34
epoch:251, iter:35
epoch:251, iter:36
epoch:251, iter:37
epoch:251, iter:38
epoch:251, iter:39
epoch:251, iter:40
epoch:251, iter:41
epoch:251, iter:42
epoch:251, iter:43
epoch:251, iter:44
epoch:251, iter:45
epoch:251, iter:46
epoch:251, iter:47
epoch:251, iter:48
epoch:251, iter:49
epoch:251, iter:50
epoch:251, iter:51
epoch:251, iter:52
epoch:251, iter:53
epoch:251, iter:54
epoch:251, iter:55
epoch:251, iter:56
epoch:251, iter:57
epoch:251, iter:58
epoch:251, iter:59
epoch:251, iter:60
epoch:251, iter:61
epoch:251, iter:62
epoch:251, iter:63
epoch:251, iter:64
epoch:251, iter:65
epoch:251, iter:66
epoch:251, iter:67
epoch:251, iter:68
epoch:251, iter:69
epoch:251, iter:70
epoch:251, iter:71
epoch:251, iter:72
epoch:251, iter:73
epoch:251, i

epoch:256, iter:31
epoch:256, iter:32
epoch:256, iter:33
epoch:256, iter:34
epoch:256, iter:35
epoch:256, iter:36
epoch:256, iter:37
epoch:256, iter:38
epoch:256, iter:39
epoch:256, iter:40
epoch:256, iter:41
epoch:256, iter:42
epoch:256, iter:43
epoch:256, iter:44
epoch:256, iter:45
epoch:256, iter:46
epoch:256, iter:47
epoch:256, iter:48
epoch:256, iter:49
epoch:256, iter:50
epoch:256, iter:51
epoch:256, iter:52
epoch:256, iter:53
epoch:256, iter:54
epoch:256, iter:55
epoch:256, iter:56
epoch:256, iter:57
epoch:256, iter:58
epoch:256, iter:59
epoch:256, iter:60
epoch:256, iter:61
epoch:256, iter:62
epoch:256, iter:63
epoch:256, iter:64
epoch:256, iter:65
epoch:256, iter:66
epoch:256, iter:67
epoch:256, iter:68
epoch:256, iter:69
epoch:256, iter:70
epoch:256, iter:71
epoch:256, iter:72
epoch:256, iter:73
epoch:256, iter:74
epoch:256, iter:75
epoch:256, iter:76
epoch:256, iter:77
epoch:256, iter:78
epoch:256, iter:79
epoch:256, iter:80
epoch:256, iter:81
epoch:256, iter:82
epoch:256, i

epoch:261, iter:40
epoch:261, iter:41
epoch:261, iter:42
epoch:261, iter:43
epoch:261, iter:44
epoch:261, iter:45
epoch:261, iter:46
epoch:261, iter:47
epoch:261, iter:48
epoch:261, iter:49
epoch:261, iter:50
epoch:261, iter:51
epoch:261, iter:52
epoch:261, iter:53
epoch:261, iter:54
epoch:261, iter:55
epoch:261, iter:56
epoch:261, iter:57
epoch:261, iter:58
epoch:261, iter:59
epoch:261, iter:60
epoch:261, iter:61
epoch:261, iter:62
epoch:261, iter:63
epoch:261, iter:64
epoch:261, iter:65
epoch:261, iter:66
epoch:261, iter:67
epoch:261, iter:68
epoch:261, iter:69
epoch:261, iter:70
epoch:261, iter:71
epoch:261, iter:72
epoch:261, iter:73
epoch:261, iter:74
epoch:261, iter:75
epoch:261, iter:76
epoch:261, iter:77
epoch:261, iter:78
epoch:261, iter:79
epoch:261, iter:80
epoch:261, iter:81
epoch:261, iter:82
epoch:261, iter:83
epoch:261, iter:84
epoch:262, iter:0
epoch:262, iter:1
epoch:262, iter:2
epoch:262, iter:3
epoch:262, iter:4
epoch:262, iter:5
epoch:262, iter:6
epoch:262, iter:7
e

epoch:266, iter:50
epoch:266, iter:51
epoch:266, iter:52
epoch:266, iter:53
epoch:266, iter:54
epoch:266, iter:55
epoch:266, iter:56
epoch:266, iter:57
epoch:266, iter:58
epoch:266, iter:59
epoch:266, iter:60
epoch:266, iter:61
epoch:266, iter:62
epoch:266, iter:63
epoch:266, iter:64
epoch:266, iter:65
epoch:266, iter:66
epoch:266, iter:67
epoch:266, iter:68
epoch:266, iter:69
epoch:266, iter:70
epoch:266, iter:71
epoch:266, iter:72
epoch:266, iter:73
epoch:266, iter:74
epoch:266, iter:75
epoch:266, iter:76
epoch:266, iter:77
epoch:266, iter:78
epoch:266, iter:79
epoch:266, iter:80
epoch:266, iter:81
epoch:266, iter:82
epoch:266, iter:83
epoch:266, iter:84
epoch:267, iter:0
epoch:267, iter:1
epoch:267, iter:2
epoch:267, iter:3
epoch:267, iter:4
epoch:267, iter:5
epoch:267, iter:6
epoch:267, iter:7
epoch:267, iter:8
epoch:267, iter:9
epoch:267, iter:10
epoch:267, iter:11
epoch:267, iter:12
epoch:267, iter:13
epoch:267, iter:14
epoch:267, iter:15
epoch:267, iter:16
epoch:267, iter:17
epo

epoch:271, iter:59
epoch:271, iter:60
epoch:271, iter:61
epoch:271, iter:62
epoch:271, iter:63
epoch:271, iter:64
epoch:271, iter:65
epoch:271, iter:66
epoch:271, iter:67
epoch:271, iter:68
epoch:271, iter:69
epoch:271, iter:70
epoch:271, iter:71
epoch:271, iter:72
epoch:271, iter:73
epoch:271, iter:74
epoch:271, iter:75
epoch:271, iter:76
epoch:271, iter:77
epoch:271, iter:78
epoch:271, iter:79
epoch:271, iter:80
epoch:271, iter:81
epoch:271, iter:82
epoch:271, iter:83
epoch:271, iter:84
epoch:272, iter:0
epoch:272, iter:1
epoch:272, iter:2
epoch:272, iter:3
epoch:272, iter:4
epoch:272, iter:5
epoch:272, iter:6
epoch:272, iter:7
epoch:272, iter:8
epoch:272, iter:9
epoch:272, iter:10
epoch:272, iter:11
epoch:272, iter:12
epoch:272, iter:13
epoch:272, iter:14
epoch:272, iter:15
epoch:272, iter:16
epoch:272, iter:17
epoch:272, iter:18
epoch:272, iter:19
epoch:272, iter:20
epoch:272, iter:21
epoch:272, iter:22
epoch:272, iter:23
epoch:272, iter:24
epoch:272, iter:25
epoch:272, iter:26
epo